In [2]:
import datetime
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('left_merged.csv')

In [4]:
def choose_description(row):
    # if description is not a string, set length to zero
    try:
        length_1 = len(row['Description 1'].split())
    except AttributeError:
        length_1 = 0
    try:
        length_2 = len(row['Description 1'].split())
    except AttributeError:
        length_2 = 0
    
    # return NaN if under 10 words in longest description
    if length_1 < 10 and length_2 < 10:
        return np.nan
    
    if length_1 >= length_2:
        return row['Description 1']
    
    return row['Description 2']

In [5]:
df['Description'] = df.apply(choose_description, axis=1)

In [6]:
def combine_descriptions(row):
    desc_1 = row['Description 1']
    desc_2 = row['Description 2']
    
    # if description is not a string, set it to empty string
    if isinstance(desc_1, float):
        desc_1 = ''
    if isinstance(desc_2, float):
        desc_2 = ''
    
    # return combined descrtiption unless length is zero, then return NaN
    combined_desc = desc_1 + ' ' + desc_2
    return combined_desc if combined_desc != ' ' else np.nan

In [7]:
df['Generated Description'] = df.apply(combine_descriptions, axis=1)

In [8]:
# drop original description columns
df = df.drop(columns=['Description 1', 'Description 2'])

## Process Laboratory Results

In [9]:
data = df

In [10]:
# Drop Lab Overview Columns
data = data.drop(columns=['ana360', 'psilabs', 'sclabs'])

# Drop unnecessary data columns

data = data.drop(columns=['Moisture Content'])

In [11]:
# Consolidate THC to Total_THC
thc_cols = ['delta-9 THC-A', 'delta-9 THC', 'delta-8 THC', 'THC-A', 'THCV']
cbd_cols = ['CBD-A', 'CBD', 'CBDV', 'CBDV-A']
cbg_cols = ['delta-9 CBG-A','delta-9 CBG']
nerolidol_cols = ['cis-Nerolidol', 'trans-Nerolidol', 'trans-Nerolidol 1', 'trans-Nerolidol 2']
ocimene_cols = ['trans-Ocimene', 'beta-Ocimene']
caryophyllene_cols = ['beta-Caryophyllene', 'Caryophyllene Oxide']
pinene_cols = ['alpha-Pinene','beta-Pinene']
terpinene_cols = ['alpha-Terpinene','gamma-Terpinene']


# Copy data for transformations, calculation of total active components
tf = data[thc_cols].copy().fillna(value=0)
cf = data[cbd_cols].copy().fillna(value=0)
gf = data[cbg_cols].copy().fillna(value=0)
nf = data[nerolidol_cols].copy().fillna(value=0)
of = data[ocimene_cols].copy().fillna(value=0)
caf = data[caryophyllene_cols].copy().fillna(value=0)
pf = data[pinene_cols].copy().fillna(value=0)
tef = data[terpinene_cols].copy().fillna(value=0)

In [12]:
# Calculate total active component and similar analytes
tf['Total_THC'] = tf[['THC-A', 'delta-9 THC-A']].max(axis=1) * 0.877 + tf['delta-9 THC'] + tf['delta-8 THC'] + tf['THCV']
cf['Total_CBD'] = cf[['CBD-A', 'CBDV-A']].sum(axis=1) * 0.877 + cf['CBD'] + cf['CBDV']
gf['Total_CBG'] = gf['delta-9 CBG-A'] + gf['delta-9 CBG']
nf['Nerolidol'] = nf.sum(axis=1)
of['Ocimene'] = of.sum(axis=1)
caf['Caryophyllene'] = caf.sum(axis=1)
pf['Pinene'] = pf.sum(axis=1)
tef['Terpinene'] = tef.sum(axis=1)


# Replace zeroes with NaN
tf.Total_THC = tf.Total_THC.replace(to_replace=0, value=np.NaN)
cf.Total_CBD = cf.Total_CBD.replace(to_replace=0, value=np.NaN)
gf.Total_CBG = gf.Total_CBG.replace(to_replace=0, value=np.NaN)
nf.Nerolidol = nf.Nerolidol.replace(to_replace=0, value=np.NaN)
of.Ocimene = of.Ocimene.replace(to_replace=0, value=np.NaN)
caf.Caryophyllene = caf.Caryophyllene.replace(to_replace=0, value=np.NaN)
pf.Pinene = pf.Pinene.replace(to_replace=0, value=np.NaN)
tef.Terpinene = tef.Terpinene.replace(to_replace=0, value=np.NaN)

In [13]:
# Add summary THC and CBD columns back to original dataframe.  Drop working columns.
data = data.drop(columns=thc_cols)
data = data.drop(columns=cbd_cols)
data = data.drop(columns=cbg_cols)
data = data.drop(columns=nerolidol_cols)
data = data.drop(columns=ocimene_cols)
data = data.drop(columns=caryophyllene_cols)
data = data.drop(columns=pinene_cols)
data = data.drop(columns=terpinene_cols)

data['Total_THC'] = tf.Total_THC
data['Total_CBD'] = cf.Total_CBD
data['Total_CBG'] = gf.Total_CBG
data['Nerolidol'] = nf.Nerolidol
data['Ocimene'] = of.Ocimene
data['Caryophyllene'] = caf.Caryophyllene
data['Terpinene'] = tef.Terpinene

In [14]:
# Clean remaining names for easier search/load operation

name_delta_map = {
    '3-Carene': 'Carene',
    'p-Cymene': 'Cymene',
    'alpha-Bisabolol': 'Bisabolol',
    'alpha-Humulene': 'Humulene',
    'beta-Myrcene': 'Myrcene',
    'delta-Limonene': 'Limonene',
}

data = data.rename(columns=name_delta_map)

In [15]:
# Compare THC reported by users/site and directly from labs, using lab value where possible

data['Total_THC'] = data['Total_THC'].fillna(value=data['THC Percent']*100)

# Drop original THC Percent column

data = data.drop(columns='THC Percent')

In [17]:
# Drop strain repeat column
data = data.drop(columns='strain')

In [18]:
data.head()

,Strain,Type,Percent Indica,Percent Sativa,Generated Description,Flavor,Effects,Rating,Carene,Camphene,...,Cymene,CBN,CBC,Description,Total_THC,Total_CBD,Total_CBG,Nerolidol,Caryophyllene,Terpinene
0,sugar-cane,hybrid,0.40,0.60,Sugar Cane is a rare slightly sativa dominant ...,Earthy Sweet Candy Grape Spicy Fruity Herbal P...,Body High Cerebral Creative Energizing Relaxin...,NaN,NaN,NaN,...,NaN,NaN,NaN,Sugar Cane is a rare slightly sativa dominant ...,20.000000,NaN,NaN,NaN,NaN,NaN
1,mac1,hybrid,0.50,0.50,"MAC 1, also known as “Miracle Alien Cookies X1...",Sweet Diesel Sour Spicy Herbal Pungent,Creative Euphoria Happy Motivation Relaxing Up...,NaN,NaN,NaN,...,NaN,NaN,NaN,"MAC 1, also known as “Miracle Alien Cookies X1...",21.500000,NaN,NaN,NaN,NaN,NaN
2,chemdawg,hybrid,0.55,0.45,With a near-even balance between sativa and in...,Earthy Pungent Chemical Diesel Pine Diesel Ear...,Cerebral Creative Euphoria Happy Relaxing Cere...,4.3,NaN,NaN,...,NaN,0.070,0.069000,With a near-even balance between sativa and in...,18.160529,0.239965,0.876875,NaN,0.812522,NaN
3,jack-herer,sativa,NaN,NaN,Jack Herer is easily one of the best-known str...,Earthy Sweet Spicy Herbal Lemon Pine Woody Ear...,Body High Cerebral Creative Euphoria Happy Bod...,4.4,0.023,0.003333,...,0.01,0.082,0.046667,Jack Herer is easily one of the best-known str...,15.461573,0.206155,0.962574,NaN,0.656769,0.034
4,nerds,hybrid,0.50,0.50,"Nerds, also known as “Nerdz,” is an evenly bal...",Earthy Sweet Grape Spicy Herbal Fruity Berry W...,Cerebral Creative Euphoria Focus Relaxing Cere...,NaN,NaN,NaN,...,NaN,NaN,NaN,"Nerds, also known as “Nerdz,” is an evenly bal...",15.500000,NaN,NaN,NaN,NaN,NaN


In [19]:
# save to csv file
now = datetime.datetime.now()
path = '../processed_data/generated-descriptions-' + now.strftime("%Y-%m-%d %H:%M") + '.csv'
data.to_csv(path)